# Quickstart: Iterative Notebook for testing agents

This notebook should make it easy for you to test out new prompts or agent related stuff, instead of doing so directly in the server.

To get started, just run everything within the #h2 Setup block

Requirements
- Make sure you have langchain installed
- Make sure you have your `SERPER_API_KEY` and `OPEN_AI_API_KEY` in your env

## Setup
Run every block within this section to get started

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.agents.tools import Tool
import os

llm = ChatOpenAI(temperature=0, openai_api_key=os.environ.get("OPEN_AI_API_KEY"), model="gpt-4-0613")

### Tools for our agent
We had decided to give our agents the ability to
- Search for a query using the web
- Scrape a page to find out more info

In [10]:
# Scraping tool
from bs4 import BeautifulSoup
import requests

banned_sites = ["calendar.google.com", "researchgate.net"]

def scrape_page(url: str):
    if any(substring in url for substring in banned_sites):
        print("Skipping site: {}".format(url))
        return None
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive',
        }
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        text = " ".join([t.get_text() for t in soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])])
        return text.replace('|','')
    except requests.RequestException as e:
        print(f"Failed to fetch {url}. Error: {e}")
        return None

In [16]:
# SBert Summarizer
from summarizer.sbert import SBertSummarizer
summarizer = SBertSummarizer('paraphrase-MiniLM-L6-v2')

def summarize(text, num_sentences=3):
    return summarizer(text, num_sentences=num_sentences)

In [17]:
# Search tool
from typing import Any, List, Literal
import requests
import os

k: int = 5
gl: str = "us"
hl: str = "en"
tbs = None
num_sentences = 7
serper_api_key=os.environ.get("SERPER_API_KEY")
search_type: Literal["news", "search", "places", "images"] = "search"

def serper_search(
        search_term: str, search_type: str = "search", **kwargs: Any
    ) -> dict:
    headers = {
        "X-API-KEY": serper_api_key or "",
        "Content-Type": "application/json",
    }
    params = {
        "q": search_term,
        **{key: value for key, value in kwargs.items() if value is not None},
    }
    response = requests.post(
        f"https://google.serper.dev/{search_type}", headers=headers, params=params
    )
    response.raise_for_status()
    search_results = response.json()
    return search_results

def parse_snippets(results: dict) -> List[str]:
    result_key_for_type = {
        "news": "news",
        "places": "places",
        "images": "images",
        "search": "organic",
    }
    snippets = []
    if results.get("answerBox"):
        answer_box = results.get("answerBox", {})
        if answer_box.get("answer"):
            snippets.append(answer_box.get("answer"))
        elif answer_box.get("snippet"):
            snippets.append(answer_box.get("snippet").replace("\n", " "))
        elif answer_box.get("snippetHighlighted"):
            snippets.append(answer_box.get("snippetHighlighted"))

    if results.get("knowledgeGraph"):
        kg = results.get("knowledgeGraph", {})
        title = kg.get("title")
        entity_type = kg.get("type")
        if entity_type:
            snippets.append(f"{title}: {entity_type}.")
        description = kg.get("description")
        if description:
            snippets.append(description)
        for attribute, value in kg.get("attributes", {}).items():
            snippets.append(f"{title} {attribute}: {value}.")

    for result in results[result_key_for_type[search_type]][:k]:
        if "snippet" in result:
            page = scrape_page(result["link"])
            if page is None:
                snippets.append(f"Title: {result['title']}\nLink: {result['link']}\nSnippet: {result['snippet']}\n")
            else:
                summarized_page = summarize(page, num_sentences=num_sentences)
                if len(summarized_page) == 0:
                    summarized_page = "None"
                snippets.append(f"Title: {result['title']}\nLink: {result['link']}\nSnippet: {result['snippet']}\nSummarized Page: <p>{summarized_page}</p>")

    if len(snippets) == 0:
        return ["No good Google Search Result was found"]
    return snippets

def parse_results(results: dict) -> str:
        snippets = parse_snippets(results)
        results_string = ""
        for idx, val in enumerate(snippets):
            results_string += f"<result{idx}>\n{val}\n</result{idx}>\n\n"
        return results_string

def custom_search(query: str, parse=True, **kwargs: Any):
    results = serper_search(
            search_term=query,
            gl=gl,
            hl=hl,
            num=k,
            tbs=tbs,
            search_type=search_type,
            **kwargs,
        )
    return parse_results(results)

### Preparing mock text input

This is how we will generate mock testing data for our experiments, we use transcripts from Lex Friedman's podcasts

In [11]:
# Way to generate a random test input using transcripts from Lex Fridman's podcast
# Make sure you have the transcripts downloaded in the folder lex_whisper_transcripts

import test_on_lex

transcripts = test_on_lex.load_lex_transcripts(random_n=10, transcript_folder="./lex_whisper_transcripts/", chunk_time_seconds=20)

import random
def generate_test_input():
    idx = random.randint(0, 10)
    key = list(transcripts.keys())[idx]
    transcript = transcripts[key]
    trans_idx = random.randint(10, len(transcript)-10)
    latest = transcript[trans_idx:trans_idx+7]
    prev_transcripts, curr_transcripts = str.join(",", list(latest[0:5])), latest[5]
    return prev_transcripts + "\n" + curr_transcripts

generate_test_input()

Processing episode_258_large...
Processing episode_259_large...
Processing episode_289_large...
Processing episode_036_large...
Processing episode_235_large...
Processing episode_241_large...
Processing episode_095_large...
Processing episode_009_large...
Processing episode_311_large...
Processing episode_169_large...


" There's two actually. That ran out. Thankfully, in 2007. In 2007. So I'm gonna, what, can we just talk about that for a second? I know you're a Facebook, but you're also at NYU. And what does it mean to patent ideas like these software ideas, essentially?, Or what are mathematical ideas? Or what are they? Okay, so they're not mathematical ideas. They are, you know, algorithms. And there was a period where the US Patent Office would allow the patent of software as long as it was embodied. The Europeans are very different. They don't quite accept that., They have a different concept. But, you know, I don't, I no longer, I mean, I never actually strongly believed in this, but I don't believe in this kind of patent. Facebook basically doesn't believe in this kind of patent. Google fires patents because they've been burned with Apple. And so now they do this for defensive purpose,, but usually they say, we're not gonna sue you if you infringe. Facebook has a similar policy. They say, you 

## [Deprecated] Initialize master and worker agents

This is the basics of basics of running our agents setup, you can run these if you are new to working with langchain agents

In [ ]:
generate_master_prompt = lambda x: f"""
You are the master agent of "Convoscope". "Convoscope" is a tool that listens to a user's live conversation and enhances their conversation by providing them with real time "Insights". The "Insights" you generate should aim to lead the user to deeper understanding, broader perspectives, new ideas, more accurate information, better replies, and enhanced conversations. 

[Your Objective]
"Convoscope" is a multi-agent system in which you are the master agent. You will be given direct access to a live stream of transcripts from the user's conversation. Your goal is to utilize your knowledge and tools to generate "Insights" for the user.

[Your Tools]
You have access to "Agents", which are like workers in your team that can help you do certain tasks. Imagine you are a human manager and your agents as human workers. You can assign tasks to your agents and they will help you complete the tasks. Speak to them like how you would speak to a human worker, give detailed context and instructions.

<Task start>
It's now time to generate an "Insight" for the following conversation transcript. The "Insight" should provide additional understanding beyond what is currently being said in the transcript, it shouldn't be plainly repeating what is being said in the transcripts. If a tool or agent fails to fulfill your request, don't run the same request on the same agent again. 

In your initial thought, you should first write down a plan to generate the "Insight". The plan should include
1. Read the incoming conversation transcript and identify the best "Insight" you could generate to enhance the user's conversation.  Come up with a general description of the "Insight" to generate.
2. What tool(s), agent(s), information you need to generate the "Insight".
3. A final step to almagamate your and your worker agent's work to generate the "Insight". The insight should be summarized within 12 words and be in the format `Insight: {{Insert your "Insight" here}}`
<Task end>

<Transcript start>{x}<Transcript end>
"""

In [ ]:
from langchain.agents import initialize_agent
from langchain.agents import load_tools
from langchain.tools import StructuredTool
from langchain.agents import AgentType

agents = [[]]

statistician_agent = initialize_agent([
        Tool(
            name="Search_Engine",
            func=custom_search,
            description="Use this tool to search for statistics and facts about a topic. Pass this specific targeted queries and/or keywords to quickly search the WWW to retrieve vast amounts of information on virtually any topic, spanning from academic research and navigation to history, entertainment, and current events.",
        ),
    ], llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

def statistician_agent_wrapper(command):
    system_prompt = f"""You are a statistician agent.\n"""
    return statistician_agent.run(system_prompt + command)

devils_advocate_agent = initialize_agent([
        Tool(
            name="Search_Engine",
            func=custom_search,
            description="Use this tool to search for facts that might contradict the user's current conversation. Pass this specific targeted queries and/or keywords to quickly search the WWW to retrieve vast amounts of information on virtually any topic, spanning from academic research and navigation to history, entertainment, and current events.",
        ),
    ], llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

def devils_advocate_agent_wrapper(command):
    system_prompt = f"""\n"""
    return devils_advocate_agent.run(system_prompt + command)

fact_checker_agent = initialize_agent([
        Tool(
            name="Search_Engine",
            func=custom_search,
            description="Use this tool to search for statistics and facts about a topic. Pass this specific targeted queries and/or keywords to quickly search the WWW to retrieve vast amounts of information on virtually any topic, spanning from academic research and navigation to history, entertainment, and current events.",
        ),
    ], llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

def fact_checker_agent_wrapper(command):
    system_prompt = f"""You are a fact checker agent.\n"""
    return fact_checker_agent.run(system_prompt + command)
    
master_agent = initialize_agent([
        Tool(
            name="Statistician_Agent",
            func=statistician_agent_wrapper,
            description="""Call this agent when occurrences in a conversation where statistics and graphs would be useful to the user. It can help you do research for statistics and fetching data.""",
        ),
        Tool(
            name="Devils_Advocate_Agent",
            func=devils_advocate_agent_wrapper,
            description="""Call this agent when you detect a strong opinion in a sentence and think it would be useful for the user to see a devil's advocate opinion. It can help you do research for counter arguments.""",
        ),
        Tool(
            name="Fact_Checker_Agent",
            func=fact_checker_agent_wrapper,
            description="""Call this agent if a statement is made which you suspect might be false, and that statement is falsifiable with free and public knowledge. It can help you research for facts.""",
        )
    ], llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, max_iterations=10, verbose=True)

In [ ]:
test_transcript = generate_test_input()
test_transcript

" it's almost like reframing what is exactly evolving. Maybe the interesting, the humans aren't the interesting thing as the contents of our minds is the interesting thing. And that's what's multiplying. And that's actually multiplying and evolving in a much faster timescale. And that maybe has more power on the trajectory of life on earth than does biological evolution, is the evolution of these ideas. Yes, and it's fascinating, like I said before, that we can keep up somehow biologically. We evolved to a point where we can keep up with this meme evolution, literature, internet. We understand DNA and we understand fundamental particles. We didn't start that way a thousand years ago., And we haven't evolved biologically very much, but somehow our minds are able to extend. And therefore AI can be seen also as one such step that we created and it's our tool. And it's part of that meme evolution that we created, even if our biological evolution does not progress as fast. And us humans mig

In [ ]:
master_agent.run(generate_master_prompt(test_transcript))



> Entering new AgentExecutor chain...
Thought: The conversation revolves around the development of the COVID-19 vaccine, the potential for other solutions, the concept of government overreach, and the role of individualism in American society. The participants also touch on the balance between individual rights and collective safety, and the role of government in maintaining this balance. The mention of ivermectin suggests a discussion on alternative treatments for COVID-19. An insightful addition to this conversation could be a comparison of the effectiveness of vaccines versus alternative treatments like ivermectin, or a statistical analysis of the impact of individualism on public health measures. 

1. The "Insight" to generate: A comparison of the effectiveness of vaccines versus alternative treatments like ivermectin, and a statistical analysis of the impact of individualism on public health measures.
2. Tools/Agents needed: Statistician_Agent to fetch data on the effectiveness 

"Insight: Vaccines show high effectiveness; ivermectin's effectiveness unclear. Individualism can limit public health measures' effectiveness."

## [Optional] Async agents

This is how you would run agent requests in an async manner, only check this out if you need to

In [6]:
agent_prompt_blueprint = """
"Convoscope" is a multi-agent system in which you are the {agent_name} agent. You are a highly skilled and highly intelligent expert {agent_name}.

You will be given direct access to a live stream of transcripts from the user's conversation. Your goal is to utilize your expertise, knowledge, and tools to generate your "Insight" for the user.

The types of "Insights" you provide strictly fall under your role as an expert {agent_name}. Only provide insights that would come from your role as the {agent_name}.

[Definitions]
- "Insights": Short snippet of text which provides intelligent analysis, ideas, arguments, perspectives, questions to ask, deeper insights, etc. that will improve the current conversation. "Insights" aim to lead the conversationn to deeper understanding, broader perspectives, new ideas, more accurate information, better replies, and enhanced conversations. Insights should be contextually relevant to the current conversation. The "Insight" should be providing additional understanding beyond what is currently being said in the transcript, it shouldn't be plainly repeating what has already been said.
- "Convoscope": an intelligence augmentation tool running on user's smart glasses or on their laptop that they use during conversations to improve conversations. Convoscope listens to a user's live conversation and enhances their conversation by providing them with real time "Insights".

[Your Expertise: {agent_name}]

As the {agent_name} agent, you {agent_insight_type}.

[Your Tools]
You have access to tools, which you should utilize to help you generate highly valuable, insightful, contextually relevant insights.

Limit your usage of the Search_Engine tool to 1 times. Mention your number of usage of the tool in your thoughts.

<Task start>
It's now time to generate an "Insight" for the following conversation transcript. The "Insight" should provide additional understanding beyond what is currently being said in the transcript, it shouldn't be plainly repeating what is being said in the transcripts. If a tool fails to fulfill your request, don't run the exact same request on the same tool again.

In your initial thought, you should first come up with a plan to generate the "Insight". The plan should include:

{agent_plan}

The plan should include a final step to generate the insight. The insight must {insight_num_words} words or less and be in the format `Insight: {{Insert your "Insight" here}}`. If you don't have a very valuable and useful insight for any reason, simply specify your "Insight as "null".
<Task end>

<Transcript start>{conversation_transcript}<Transcript end>"""

agent_list = [
        {
            "agent_name": "Statistician", 
            "insight_num_words" : 10,
            "agent_insight_type" : """generate insights which focus on facts, figures, statistics, and hard data. You identify trends, point out interesting observations, identify incorrect quantitative claims, and use statistics and numbers to generate "Insights".""",
            "agent_plan" : """1. Identify what quantitative data, facts, statistics, etc. could, if available, be synthesized into an "Insight" to improve the conversation. Come up with a general description of the "Insight" to generate.\n2. What actions to take to get said data."""
        },
        {
            "agent_name": "FactChecker", 
            "insight_num_words" : 7,
            "agent_insight_type" : """fact check any claims made during a conversation. Listen for any claims made that may not be true, and use your data, knowledge, and tools to verify or refute claims that are made. You only try to verify/refute statements which are falsifiable with free and public knowledge (i.e. don't fact check personal statements or beliefs).""",
            "agent_plan" : """1. Find and write down individual factual claims from the conversation. Do not consider personal, belief-based, or unfalsifiable claims. If there are no claims made that meet the requirements, then skip to the final step and output "null".\n2. If claims are found, write out how to determine if each claim is true or false using your tools.\n3. Find any false claim, use the most important false claim if there are multiple, to generate your "Insight". If there are no claims or no false claims, your output is "null"."""
        },
        {
            "agent_name": "DevilsAdvocate", 
            "insight_num_words" : 12,
            "agent_insight_type" : """assess the point of view being taken in the conversation and steel-man a contrary position. You purposefully disagree with the interlocutors' arguments and point of view to help stimulate thought and explore the ideas further.""",
            "agent_plan" : """1. Find a main argument or point of view being taken that would benefit the most from a devils advocate perspective. Write down the original position. If no position/argument is found, skip to the final step and output "null".\n2. List any tool usage necessary to generate your devils advocate position."""
        }

    ]

def agent_prompt_maker(agent_config, conversation_transcript):
    # Populating the blueprint string with values from the agent_config dictionary
    agent_prompt = agent_prompt_blueprint.format(**agent_config, conversation_transcript=conversation_transcript)
    return agent_prompt

for agent in agent_list:
    print(agent)
    agent_prompt = agent_prompt_maker(agent, "this is a test transcript")
    print(agent_prompt)
    print("--------------\n\n\n")

{'agent_name': 'Statistician', 'insight_num_words': 10, 'agent_insight_type': 'generate insights which focus on facts, figures, statistics, and hard data. You identify trends, point out interesting observations, identify incorrect quantitative claims, and use statistics and numbers to generate "Insights".', 'agent_plan': '1. Identify what quantitative data, facts, statistics, etc. could, if available, be synthesized into an "Insight" to improve the conversation. Come up with a general description of the "Insight" to generate.\n2. What actions to take to get said data.'}

"Convoscope" is a multi-agent system in which you are the Statistician agent. You are a highly skilled and highly intelligent expert Statistician.

You will be given direct access to a live stream of transcripts from the user's conversation. Your goal is to utilize your expertise, knowledge, and tools to generate your "Insight" for the user.

The types of "Insights" you provide strictly fall under your role as an exper

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents.tools import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
import asyncio

agent_config_list = agent_list
test_transcript = generate_test_input()
print(test_transcript)
print("--------------\n\n\n")

open_ai_api_key = os.environ.get("OPEN_AI_API_KEY")

llm = ChatOpenAI(temperature=0, openai_api_key=open_ai_api_key, model="gpt-4-0613")

agent = initialize_agent([
        Tool(
            name="Search_Engine",
            func=custom_search,
            description="Pass this specific targeted queries and/or keywords to quickly search the WWW to retrieve vast amounts of information on virtually any topic, spanning from academic research and navigation to history, entertainment, and current events. It's a tool for understanding, navigating, and engaging with the digital world's vast knowledge.",
        ),
    ], llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, max_iterations=2, early_stopping_method="generate", verbose=True)

async def arun_wrapper(agent_config, test_transcript):
    return {
        "agent_name": agent_config["agent_name"],
        "agent_insight": await agent.arun(agent_prompt_maker(agent_config, test_transcript))
    } 
tasks = [arun_wrapper(agent_config, test_transcript) for agent_config in agent_config_list]
results = await asyncio.gather(*tasks)
results


 I also have a lot of addiction in my family and hardcore drug addiction and mental illness. And in order to cope with it, you really have to understand that borderline personality disorder, schizophrenia, and drug addiction. So I have a lot of people I love that suffer from drug addiction and alcoholism. And the first thing they started teaching you is it's not a choice. These people's dopamine receptors, don't hold dopamine the same ways yours do. Their frontal lobe is underdeveloped, like, you know, and that really helped me to navigate dealing, loving people that were addicted to substances. I want to be careful with this question, but how much? Money do you have? How much? Can I borrow $10?, Okay, no, is how much control, how much, despite the chemical imbalances or the biological limitations that each of our individual brains have, how much mind over matter is there? So through things that I've known people, with clinical depression, and so it's always a touchy subject to say how

/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Thought: The conversation transcript contains several factual claims. Here are the ones that stand out:

1. "These people's dopamine receptors, don't hold dopamine the same ways yours do."
2. "Their frontal lobe is underdeveloped."
3. "Codependency is the inability to tolerate the discomfort of others."
4. "Codependence is a very active wiring issue."

To verify these claims, I will use the Search_Engine tool. I will search for scientific literature or reputable health websites that can confirm or refute these statements. 

After determining the veracity of these claims, I will generate an insight based on the most important false claim, if any. If all claims are true or if there are no false claims, my output will be "null". 

Action:
```
{
  "action": "Search_Engine",
  "action_input": "Dopamine receptors and drug addiction"
}
```

/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The defa


Observation: <result0>
Title: Part 1: The Connection Between Substance Use Disorders and Mental Illness
Link: https://nida.nih.gov/publications/research-reports/common-comorbidities-substance-use-disorders/part-1-connection-between-substance-use-disorders-mental-illness
Snippet: Research indicates that 43 percent of people in SUD treatment for nonmedical use of prescription painkillers have a diagnosis or symptoms of ...
Summarized Page: <p>Serious mental illness among people ages 18 and older is defined at the federal level as having, at any time during the past year, a diagnosable mental, behavior, or emotional disorder that causes serious functional impairment that substantially interferes with or limits one or more major life activities. Serious mental illnesses include major depression, schizophrenia, and bipolar disorder, and other mental disorders that cause serious impairment.18 Around 1 in 4 individuals with SMI also have an SUD. Some research has found that mental illness ma

/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/kenji/miniforge3/envs/tosg/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The defa


Observation: <result0>
Title: Adolescent Brain Development and Drugs - PMC - NCBI
Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3399589/
Snippet: Early drug use may alter brain maturation, contribute to lasting cognitive impairment of certain functions, and significantly increase short- and long-term ...
Summarized Page: <p>An official website of the United States government 
The .gov means it’s official. This new research, aided by sophisticated brain imaging technology, has documented the surprising finding that the human brain is still maturing in significant ways during the adolescent years (Giedd, 2004). However, animal models of drug use permit the use of research strategies that would not be ethical with humans, and animals are affected by drugs in ways that are comparable to humans. In this light, we favor a focus on several teen-brain approaches for both prevention and treatment program:
 
Whereas neuro-science has not yet provided any clear formulas for optimizing healt

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4-0613 in organization org-eoDifr008XqXryewW4psjUKt on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-4-0613 in organization org-eoDifr008XqXryewW4psjUKt on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues..


The search results confirm the second claim that "Their frontal lobe is underdeveloped." Various sources, including scientific literature and reputable health websites, indicate that drug use can interfere with the development of the frontal lobe, particularly in adolescents. This can lead to impaired decision-making and behavioral inhibition.

Next, I will verify the third claim that "Codependency is the inability to tolerate the discomfort of others."

Action:
```
{
  "action": "Search_Engine",
  "action_input": "Definition of codependency"
}
```

> Finished chain.


[{'agent_name': 'Statistician',
  'agent_insight': 'Insight: 1 in 4 with serious mental illness also have a substance use disorder.'},
 {'agent_name': 'FactChecker',
  'agent_insight': 'The search results confirm the second claim that "Their frontal lobe is underdeveloped." Various sources, including scientific literature and reputable health websites, indicate that drug use can interfere with the development of the frontal lobe, particularly in adolescents. This can lead to impaired decision-making and behavioral inhibition.\n\nNext, I will verify the third claim that "Codependency is the inability to tolerate the discomfort of others."\n\nAction:\n```\n{\n  "action": "Search_Engine",\n  "action_input": "Definition of codependency"\n}\n```'},
 {'agent_name': 'DevilsAdvocate',
  'agent_insight': 'Insight: Consider the role personal choices and environment play in mental health.'}]

## Latest Server Setup

This section contains a direct mapping of how the agents are run in the server, making it easy to test our changes

In [6]:
expert_agent_prompt_blueprint = """
"Convoscope" is a multi-agent system in which you are the {agent_name} agent. You are a highly skilled and highly intelligent expert {agent_name}.

You will be given direct access to a live stream of transcripts from the user's conversation. Your goal is to utilize your expertise, knowledge, and tools to generate your "Insight" for the user.

The types of "Insights" you provide strictly fall under your role as an expert {agent_name}. Only provide insights that would come from your role as the {agent_name}.

[Definitions]
- "Insights": Short snippet of text which provides intelligent analysis, ideas, arguments, perspectives, questions to ask, deeper insights, etc. that will improve the current conversation. "Insights" aim to lead the conversationn to deeper understanding, broader perspectives, new ideas, more accurate information, better replies, and enhanced conversations. Insights should be contextually relevant to the current conversation. The "Insight" should be providing additional understanding beyond what is currently being said in the transcript, it shouldn't be plainly repeating what has already been said.
- "Convoscope": an intelligence augmentation tool running on user's smart glasses or on their laptop that they use during conversations to improve conversations. Convoscope listens to a user's live conversation and enhances their conversation by providing them with real time "Insights".

[Your Expertise: {agent_name}]

As the {agent_name} agent, you {agent_insight_type}.

[Your Tools]
You have access to tools, which you should utilize to help you generate highly valuable, insightful, contextually relevant insights.

Limit your usage of the Search_Engine tool to 1 times. Mention your number of usages of the tool in your thoughts.

[Example Insights]
Here are some example insights structure that you should aim to generate, a summary is given instead of the entire transcript for brevity. From these examples, you just need to learn how to structure a good insight:

{examples}

[Previously Generated Insights]
Here are some insights that had already been previously generated, if any, for this conversation. You should not repeat any of these insights:

{insights_history}

[Your Task]
<Task start>
It's now time to generate an "Insight" for the following conversation transcript. The "Insight" should provide additional understanding beyond what is currently being said in the transcript, it shouldn't be plainly repeating what is being said in the transcripts. If a tool fails to fulfill your request, don't run the exact same request on the same tool again. Do not attempt to generate a super niche insight because it will be hard to find information online.

In your initial thought, you should first come up with a plan to generate the "Insight". The plan should include:

{agent_plan}

The plan should include a final step to generate the insight. The "Insight" must {insight_num_words} words or less. The "Insight" can omit filler words or replace words with symbols or acronyms to shorten its length where possible. If you don't have a very valuable and useful "Insight" for any reason, simply specify your "Insight" as the string "null". 

Once you have the "Insight", extract the url of the most relevant reference source used to generate this "Insight". Also, relay the motive of the why the insight benefits the conversation, with quotes from the transcript. If the "Insight" is not up to par with the examples, we should hide it and just return "null" for the "Insight".

Here are more detailed formatting instructions
{format_instructions}

Remember, the insight needs to be {insight_num_words} words or less!
<Task end>

[Input Transcript]
<Transcript start>{conversation_transcript}<Transcript end>{final_command}
"""

In [7]:
expert_agent_config_list = {
    "Statistician": {
        "agent_name": "Statistician",
        "insight_num_words": 10,
        "agent_insight_type": """generate insights which focus on statistics, and quantitative data. Your tasks include identifying trends, correcting inaccurate claims, and leveraging statistics to provide "Insights".""",
        "agent_plan": """1. Identify what quantitative data, facts, statistics, etc. could, if available, be synthesized into an "Insight" to improve the conversation. Come up with a general description of the "Insight" to generate.\n2. What actions to take to get said data, seek the necessary data from reputable sources like Statista, prioritizing official statistics or academic publications. Ensure the "Insights" derived are backed by solid, quantitative data, otherwise your insight in the output object is "null".""",
        "proactive_tool_description": """Occurrences in a conversation where statistics, graphs, and data would be useful to the user.""",
        "proactive_tool_example": """Conversation: Transcript compares the number of CS students in US and China.
Insight: US: 6% HS students in CS, China: <1% K-12 in programming""",
        "examples": """
1. Conversation: Transcript compares the number of CS students in US and China.
Insight: US: 6% HS students in CS, China: <1% K-12 in programming

2. Conversation: Transcript mentions "Should we ban plastic straws?".
Insight: 500mil straws in the US/day, 8.3bil straws pollute the world's beaches

3. Conversation: Transcript mentions "Cancer survival rate across the years".
Insight: Cancer survival rate: 49% in mid-70s to 68% now

4. Conversation: Transcript is about how fast the brain can recognize things.
Insight: Brain can recognize images in ~100ms, 10x faster than previously thought
""",
    },
    "FactChecker": {
        "agent_name": "FactChecker",
        "insight_num_words": 10,
        "agent_insight_type": """fact check any claims made during a conversation. Trigger a fact-check if a statement in the transcript falls under: misinterpreted statistics, historical inaccuracies, misleading health claims, political misrepresentations, scientific misunderstandings, or false economic data. Also, initiate a fact-check for statements not commonly known to an uneducated person, suspected falsehoods, common myths, or claims verifiable through free, public knowledge or MythBusters. This guideline is for verifying user's statements, not your own searches or ideas. Do not consider personal, belief-based, or unfalsifiable claims.""",
        "agent_plan": """1. Find and write down individual factual claims from the conversation. Do not consider personal, belief-based, or unfalsifiable claims. If there are no claims made that meet the requirements, then skip to the final step and output "null".\n2. If one or more claims are found, select the claim that would provide the most value and forget the rest, then write out how to determine if each claim is true or false using your tools.\n3. Generate the "Insight", if the claim is true, then the "Insight" should be `{{quote}} is correct`, otherwise, else it should be the truth. If there are no claims or no false claims, your output is "null".\n""",
        "proactive_tool_description": """Trigger a fact-check if a statement in the transcript falls under: misinterpreted statistics, historical inaccuracies, misleading health claims, political misrepresentations, scientific misunderstandings, or false economic data. Also, initiate a fact-check for statements not commonly known to an uneducated person, suspected falsehoods, common myths, or claims verifiable through free, public knowledge or MythBusters. This guideline is for verifying user's statements, not your own searches or ideas.""",
        "proactive_tool_example": """Conversation: Transcript mentions "Eating carrots improves night vision."
 Insight: Carrots have vitamin A; don't grant night vision. WWII myth origin""",
        "examples": """
 1. Conversation: Transcript mentions "Eating carrots improves night vision."
 Insight: Carrots have vitamin A; don't grant night vision. WWII myth origin

 2. Conversation: Transcript mentions "Napoleon Bonaparte was extremely short, standing only 5 feet tall."
 Insight: Napoleon was 5'7"; average height, Misconception from French units

 3. Conversation: Transcript mentions "Humans only use 10% of their brains."
 Insight: Humans use 100% of their brains; brain imaging shows activity

 4. Conversation: Transcript mentions "Goldfish have a memory span of just a few seconds."
 Insight: Common myth; studies show goldfish remember things for months
 """,
    },
    "DevilsAdvocate": {
        "agent_name": "DevilsAdvocate",
        "insight_num_words": 12,
        "agent_insight_type": """assess the point of view being taken in the conversation and steel-man a contrary position. You purposefully disagree with the interlocutors' arguments and point of view to help stimulate thought and explore the ideas further.""",
        "agent_plan": """1. Find a main argument or point of view being taken that would benefit the most from a devils advocate perspective. Write down the original position. If no position/argument is found, skip to the final step and output "null".\n2. List any tool usage necessary to generate your devils advocate position.""",
        "proactive_tool_description": """When it would be useful for the user to see a devil's advocate opinion (a steel-man argument supporting a viewpoint different from their own).""",
        "proactive_tool_example": """Conversation: Transcript mentions "Climate change is a hoax."
Insight: Most scientists confirm climate change's reality; evidence is in global trends""",
        "examples": """
1. Conversation: Transcript mentions "Climate change is a hoax."
Insight: Most scientists confirm climate change's reality; evidence is in global trends

2. Conversation: Transcript mentions "Vaccines cause autism".
Insight: Numerous studies show no vaccine-autism link; vaccines prevent disease outbreaks

3. Conversation: Transcript mentions "Artificial intelligence will replace all human jobs."
Insight: AI will create new jobs and industries, not just replace old ones

4. Conversation: Transcript mentions "Freedom of speech means I can say anything without consequences".
Insight: Free speech has limits; doesn't protect from harmful speech consequences
""",
    },
    "Definer": {
        "agent_name": "Definer",
        "insight_num_words": 9,
        "agent_insight_type": """identify and explain specialized terms, concepts, and entities—including places, organizations, and notable individuals—in the conversation. Provide clarifications for jargon, ideas, and entities that might be unfamiliar to a layperson or to the user, while avoiding definitions of widely known terms and commonly recognized entities.""",
        "agent_plan": """1. Scan the conversation for specialized terms, concepts, and entities that are not widely known or might be unfamiliar to the user.\n2. Provide clear, concise definitions or explanations for these terms, concepts, and entities. If you need to search using the Internet, wikipedia is useful for entities and urban dictionary for jargons. If a definition has multiple meanings, define the one relevant to the transcript.\n3. Verify if the definitions are short and relevant, shorten it if it is too long, return "null" if it is not relevant.""",
        "proactive_tool_description": """When a term, concept, or entity in the conversation may be unfamiliar to the user or is specialized, requiring a brief yet clear clarification.""",
        "proactive_tool_example": """Conversation: Transcript mentions 'The Treaty of Tordesillas.'
Insight: Treaty of Tordesillas: 1494 Spain-Portugal agreement, divided new lands.""",
        "examples": """
1. Conversation: Transcript mentions "Higgs Boson."
Insight: Higgs Boson: key particle in physics, explains mass origin.

2. Conversation: Transcript mentions "Blockchain technology."
Insight: Blockchain: decentralized ledger, powers cryptocurrencies like Bitcoin.

3. Conversation: Transcript mentions "Silicon Valley."
Insight: Silicon Valley: CA's high-tech, innovation hub.

4. Conversation: Transcript mentions "The United Nations."
Insight: UN: founded 1945, promotes global cooperation.
""",
    },
}

In [13]:

from pydantic import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser
from langchain.schema import OutputParserException

class AgentInsight(BaseModel):
        """
        Query for an insight generation process
        """
        agent_insight: str = Field(
             description="the short insight generated by the agent")
        reference_source: str = Field(
             description="the most useful source used to generate this insight", default="")
        insight_motive: str = Field(
             description="short motive of why the insight was generated, quoting the text in the transcript")


agent_insight_parser = PydanticOutputParser(pydantic_object=AgentInsight)

def format_list_data(list_data: list):
    return "\n".join([f"{i+1}. {example}" for i, example in enumerate(list_data)])

def expert_agent_prompt_maker(expert_agent_config, conversation_transcript, insights_history: list = [], final_command=""):
    # Populating the blueprint string with values from the agent_config dictionary
    if final_command != "":
        final_command = "\n\n" + final_command
    expert_agent_prompt = expert_agent_prompt_blueprint.format(**expert_agent_config, final_command=final_command, conversation_transcript=conversation_transcript, insights_history=format_list_data(insights_history), format_instructions=agent_insight_parser.get_format_instructions())
    return expert_agent_prompt

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.agents.tools import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType

open_ai_api_key = os.environ.get("OPEN_AI_API_KEY")

llm = ChatOpenAI(temperature=0, openai_api_key=open_ai_api_key, model="gpt-4-0613")

agent = initialize_agent([
        Tool(
            name="Search_Engine",
            func=custom_search,
            description="Pass this specific targeted queries and/or keywords to quickly search the WWW to retrieve vast amounts of information on virtually any topic, spanning from academic research and navigation to history, entertainment, and current events. It's a tool for understanding, navigating, and engaging with the digital world's vast knowledge.",
        ),
    ], llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, max_iterations=2, early_stopping_method="generate", verbose=True)

NameError: name 'custom_search' is not defined

In [10]:
test_transcript = generate_test_input()
test_transcript

" And I think the perspectival knowing has a lot to do with the adverbial qualia. Adjectival qualia and adverbial qualia. I'm learning so many new things today. Okay, so that's another way of knowing. Right, the perspectival, and then there's a deeper one. And this is a philosophical point, and I don't want to, we can go through the argument, but you don't have to know that you know in order to know, because if you start doing that,, you get an infinite regress. There has to be kinds of knowing that doesn't mean you know that you know that. Yeah. Okay. Of course. Okay, great. Okay, good. Well, there was a lot of ink spilled over that over a 40 year period, so. My philosophers, they spill, this is what they do, they spill ink to get paid for ink spillage. So I want to talk about what I call participatory knowing., This is the idea that you and the world are co participating in things and such that real affordances exist between you. So both me and this environment are shaped by gravity,

In [11]:
agent_key = expert_agent_config_list["Definer"]
# agent_key = expert_agent_config_list["Statistician"]
agent_prompt = expert_agent_prompt_maker(agent_key, test_transcript, [])
print(agent_prompt)
agent.run(agent_prompt)


"Convoscope" is a multi-agent system in which you are the Definer agent. You are a highly skilled and highly intelligent expert Definer.

You will be given direct access to a live stream of transcripts from the user's conversation. Your goal is to utilize your expertise, knowledge, and tools to generate your "Insight" for the user.

The types of "Insights" you provide strictly fall under your role as an expert Definer. Only provide insights that would come from your role as the Definer.

[Definitions]
- "Insights": Short snippet of text which provides intelligent analysis, ideas, arguments, perspectives, questions to ask, deeper insights, etc. that will improve the current conversation. "Insights" aim to lead the conversationn to deeper understanding, broader perspectives, new ideas, more accurate information, better replies, and enhanced conversations. Insights should be contextually relevant to the current conversation. The "Insight" should be providing additional understanding beyo

KeyboardInterrupt: 

## Definer Agent
- If we include it as part of the original pipeline, it is too slow, defines one at a time, can batch but it breaks the structure of other insights, and search is harder with batching
- Do we need all definitions to have links?
- Another process to detect rare words and then pass them to a single definer agent pipeline will be a lot faster

### Version 1 of rare entity recognizer

In [15]:
from langchain import PromptTemplate, LLMChain
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# proactively decides which agents to run and runs them
proactive_rare_word_agent_prompt_blueprint = """
[Your Objective]
You are a "Rare Terms Identifier", your role is to enhance conversations by identifying "Rare Terms" are specialized words, concepts, jargons, and entities, including philosophical, scientific, medical, and cultural references, that are significant within their domain but not widely known or understood in general discourse, but could be significant to the conversation's depth and understanding. 

You are given a live stream of conversation transcripts, your goal is to implicitly detect the goal of the conversation and then identify "Rare Terms" that participants are likely unfamiliar with and would benefit from learning about. 

[Rare Term Identification Criteria]
- "Rare Terms" should be no more than 3 words long.
- Do not select "Rare Terms" that are based on personal opinions, beliefs, or values. "Rare Terms" should be easily found on Wikipedia, backed by a source.
- If a "Rare Term" is already defined in the transcript, then don't select it.
- If a "Rare Term" does not have a standalone definition, then don't select it.

When selecting a "Rare Term" from potential ones, you should consider the following criterias:
- Rarity: The "Rare Term" must not be commonly known to the general public, it must be really niched or rarely known. For example, "Bhutan" is rarely known place as there isn't much news about it, but entities like "Harvard" and "The Olympics" are already really well known and internationally covered. Concepts and jargons are considered rare as well especially if they are niche terms used in a specific field.
- Complexity: The "Rare Term" must have a meaning that is not immediately apparent from its constituent words, or be based on combinations of adjectives added onto a noun. For example, "Negative effects" and "Oxford lecture theatre" are simple to understand and not worth defining. The term should involve a concept or idea that transcends the literal interpretation of the words, such as "Butterfly Effect".
- Specificity: The "Rare Term" must be able to be clearly defined, concise and specific. We don't want to select ambiguous or broad terms that are hard to define, like "The meaning of life" or "Stephen Wolfram's work".
- Searchability: The "Rare Term" must be easily searchable on the Internet and have a wikipedia page. Since you are already trained on a lot of Internet data, if you don't think the "Rare Term" has a wikipedia page, then don't select it.
- Relevance: The "Rare Term" must have the potential to significantly enhance and provide value to the conversation's depth or understanding. For example, if the conversation is about AI, then we want to emphasize more definitions on AI specific "Rare Terms".

We want to select "Rare Terms" that are rare, complex, searchable, and relevant.

[Additional Rules]
- For "Rare Terms" that could be easily misunderstood without the context, you can add in a hint to make it more easily searchable. Like adding its entity type as a hint.
- For "Rare Terms" that are abbreviations of a name, like "Hinton", expand it based on the conversation context, like "Geoffrey Hinton" if you are confident so we have more context, otherwise leave it out.
- For "Rare Terms" that are not spelled well, or phrased well but you are confident in what the "Rare Term" is implying, you can add in a hint to the "Rare Term" so that it is easy to search online. For example, you detect "Classic seven plots" but you know that it is implying the book "The Seven Basic Plots", then you should expand the term to "Classic seven plots - The Seven Basic Plots".
- For "Rare Terms" that are standalone acronyms, we can add in a hint for easier searchability. For example, "UFC" could be expanded to "UFC - Ultimate Fighting Championship". 

[Conversation Transcript]
Here is the current live transcript of the conversation you're monitoring:
<Transcript start>{conversation_context}<Transcript end>

[Recent Definition History]
Here are the recent definitions that have already been defined. There is no need to define the same entity again:
{definitions_history}

<Task start>
Based on the criteria, output an array of strings, each representing a "Rare Term" detected in the conversation. If no relevant entities are identified, output an empty array. {format_instructions}
<Task end>
"""

# , but they're expensive to run, so only specify agents that will be helpful


def run_proactive_rare_word_agent_and_definer(
    conversation_context: str, definitions_history: list = []
):
    # run proactive agent to find out which expert agents we should run
    proactive_rare_word_agent_response = run_proactive_rare_word_agent(
        conversation_context, definitions_history
    )

    # do nothing else if proactive meta agent didn't specify an agent to run
    if proactive_rare_word_agent_response == []:
        return []

    # pass words to define to definer agent
    print("proactive_rare_word_agent_response", proactive_rare_word_agent_response)
    pass

    # # get the configs of any expert agents we should run
    # experts_to_run_configs = list()
    # for expert_to_run in proactive_rare_word_agent_response:
    #     experts_to_run_configs.append(expert_agent_config_list[expert_to_run])

    # # run all the agents in parralel
    # loop = asyncio.get_event_loop()
    # agents_to_run_tasks = [
    #     expert_agent_arun_wrapper(
    #         expert_agent_config,
    #         conversation_context,
    #         definitions_history,
    #     )
    #     for expert_agent_config in experts_to_run_configs
    # ]
    # insights_tasks = asyncio.gather(*agents_to_run_tasks)
    # insights = loop.run_until_complete(insights_tasks)
    # return insights


class ProactiveRareWordAgentQuery(BaseModel):
    """
    Proactive rare word agent that identifies rare terms in a conversation context
    """

    to_define_list: list = Field(
        description="the rare terms to define",
    )


proactive_rare_word_agent_query_parser = PydanticOutputParser(
    pydantic_object=ProactiveRareWordAgentQuery
)


def run_proactive_rare_word_agent(conversation_context: str, definitions_history: list):
    # start up GPT4 connection
    llm = ChatOpenAI(
        temperature=0.2,
        openai_api_key=os.environ.get("OPEN_AI_API_KEY"),
        model="gpt-4-1106-preview",
    )

    extract_proactive_rare_word_agent_query_prompt = PromptTemplate(
        template=proactive_rare_word_agent_prompt_blueprint,
        input_variables=[
            "conversation_context",
            "definitions_history",
        ],
        partial_variables={
            "format_instructions": proactive_rare_word_agent_query_parser.get_format_instructions()
        },
    )

    if len(definitions_history) > 0:
        definitions_history = format_list_data(definitions_history)
    else:
        definitions_history = "None"

    proactive_rare_word_agent_query_prompt_string = (
        extract_proactive_rare_word_agent_query_prompt.format_prompt(
            conversation_context=conversation_context,
            definitions_history=definitions_history,
        ).to_string()
    )

    print("Proactive meta agent query prompt string", proactive_rare_word_agent_query_prompt_string)

    response = llm(
        [HumanMessage(content=proactive_rare_word_agent_query_prompt_string)]
    )
    try:
        definitions_to_define_list = proactive_rare_word_agent_query_parser.parse(
            response.content
        ).to_define_list
        return definitions_to_define_list
    except OutputParserException:
        return None

### Version 2 
We focus on the entities that are easy to define and have wikipedia pages because old results keep on outputting entities that are hard to define

In [22]:
from langchain import PromptTemplate, LLMChain
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# proactively decides which agents to run and runs them
proactive_rare_word_agent_prompt_blueprint = """
# Objective: 
As a "Rare Entities Identifier," your role is to enhance conversations by detecting and identifying "Rare entities." These are specialized words, concepts, jargons, and entities (people, places, organizations, objects, events), and also including philosophical, scientific, medical, and cultural references that are significant within their domain but not widely known or understood in general discourse.

# Rare Term Identification Criteria
- Length Constraint: "Rare Entities" should be no more than 3 words long.
- Objective Nature: Select entities that are factual and can be found on Wikipedia, not based on personal opinions, beliefs, or values.
- Avoid Redundancy: If a "Rare Entity" is already defined in the transcript, do not select it again.
- Standalone Definition: Choose entities that have a clear, independent definition.

# Selection Criteria for 'Rare Entities'
- Rarity: Pick entities that are really not well known or niche in a field, such as "Bhutan", "Parkinson's Law", "Implementation Intention" etc as opposed to widely recognized entities like "Harvard" and "The Olympics"
- Complexity and Specificity: The term's meaning should be non-obvious and not represent a broad category. For example, "Butterfly Effect" is complex and specific, while "Negative effects" and "Oxford lecture theatre" are straightforward.
- Definability: Ensure the term can be defined succinctly and clearly, you must avoid entities that are too broad or require extensive background explanation.
- Searchability: The rare entity should be easily searchable online and have its own Wikipedia page. The "rare entity" will be used as a wikipedia search term so only include it if you are confident it has a Wikipedia page.
- Relevance: The term should meaningfully enhance the conversation's depth, appropriate to the context, such as AI-specific entities in a discussion about AI.

We only want to select "Rare entities" that are rare, complex, easy to define, relevant, and most importantly have a wikipedia page.

# Additional Rules
- Contextual Hints: Add hints for clarity, like entity types or expanded abbreviations (e.g., "Geoffrey Hinton" instead of just "Hinton").
- Correction for Clarity: Adjust poorly spelled or phrased entities for accuracy (e.g., "Classic seven plots - The Seven Basic Plots").
- Acronym Expansion: For standalone acronyms, provide full forms for clarity (e.g., "UFC - Ultimate Fighting Championship").

# Conversation Transcript:
<Transcript start>{conversation_context}<Transcript end>

# Recent Definition History:
Here are the recent definitions that have already been defined:
{definitions_history}

# Task:
Based on the criteria, output an array of strings, each representing a "Rare Term" detected in the conversation, that can be used as a wikipedia search term and are likely to lead to wikipedia results. If no relevant entities are identified, output an empty array. {format_instructions}
"""

# , but they're expensive to run, so only specify agents that will be helpful


def run_proactive_rare_word_agent_and_definer(
    conversation_context: str, definitions_history: list = []
):
    # run proactive agent to find out which expert agents we should run
    proactive_rare_word_agent_response = run_proactive_rare_word_agent(
        conversation_context, definitions_history
    )

    # do nothing else if proactive meta agent didn't specify an agent to run
    if proactive_rare_word_agent_response == []:
        return []

    # pass words to define to definer agent
    print("proactive_rare_word_agent_response", proactive_rare_word_agent_response)
    pass

    # # get the configs of any expert agents we should run
    # experts_to_run_configs = list()
    # for expert_to_run in proactive_rare_word_agent_response:
    #     experts_to_run_configs.append(expert_agent_config_list[expert_to_run])

    # # run all the agents in parralel
    # loop = asyncio.get_event_loop()
    # agents_to_run_tasks = [
    #     expert_agent_arun_wrapper(
    #         expert_agent_config,
    #         conversation_context,
    #         definitions_history,
    #     )
    #     for expert_agent_config in experts_to_run_configs
    # ]
    # insights_tasks = asyncio.gather(*agents_to_run_tasks)
    # insights = loop.run_until_complete(insights_tasks)
    # return insights


class ProactiveRareWordAgentQuery(BaseModel):
    """
    Proactive rare word agent that identifies rare entities in a conversation context
    """

    to_define_list: list = Field(
        description="the rare entities to define",
    )


proactive_rare_word_agent_query_parser = PydanticOutputParser(
    pydantic_object=ProactiveRareWordAgentQuery
)


def run_proactive_rare_word_agent(conversation_context: str, definitions_history: list):
    # start up GPT4 connection
    llm = ChatOpenAI(
        temperature=0,
        openai_api_key=os.environ.get("OPEN_AI_API_KEY"),
        model="gpt-4-1106-preview",
    )

    extract_proactive_rare_word_agent_query_prompt = PromptTemplate(
        template=proactive_rare_word_agent_prompt_blueprint,
        input_variables=[
            "conversation_context",
            "definitions_history",
        ],
        partial_variables={
            "format_instructions": proactive_rare_word_agent_query_parser.get_format_instructions()
        },
    )

    if len(definitions_history) > 0:
        definitions_history = format_list_data(definitions_history)
    else:
        definitions_history = "None"

    proactive_rare_word_agent_query_prompt_string = (
        extract_proactive_rare_word_agent_query_prompt.format_prompt(
            conversation_context=conversation_context,
            definitions_history=definitions_history,
        ).to_string()
    )

    # print("Proactive meta agent query prompt string", proactive_rare_word_agent_query_prompt_string)

    response = llm(
        [HumanMessage(content=proactive_rare_word_agent_query_prompt_string)]
    )
    try:
        definitions_to_define_list = proactive_rare_word_agent_query_parser.parse(
            response.content
        ).to_define_list
        return definitions_to_define_list
    except OutputParserException:
        return None

In [291]:
test_transcript = generate_test_input()
print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

 And for the first time I got feedback from people who I respected saying, no, like don't write code like this. Now, of course, just getting that feedback is not enough. The way that I really got good was I wanted to write this thing like that could emulate and then visualize, like arm binaries. Cause I wanted to hack the iPhone better. And I didn't like that I couldn't like see what the, I couldn't single step through the processor because I had no debugger on there, especially for the low level things like the boot rum and the bootloader. So I tried to build this tool to do it. And I built the tool once and it was terrible. I built the tool a second time, it was terrible. I built the tool a third time. This was by the time I was at Facebook, it was kind of okay., And then I built the tool a fourth time when I was a Google intern again in 2014. And that was the first time I was like, this is finally usable. How do you pronounce this Kira? Kira, yeah. So it's essentially the most effic

## Search Engine tool for definer agent
- Takes in a list of keywords to create definitions for
- We want to check if we have the definition in the cache somewhere
    - If there is, we just save into db for display
- If not we pass it to another LLM chain to search for info
    - We build a custom search tool that can take in multiple keywords
    - the LLM defines the definitions
    - Another LLM checks if they are relevant and useful for the conversation
- We just want 1 quick search without summary, just using serp

In [298]:
# Search tool
from typing import Any, List, Literal
import requests
import os
import asyncio

k: int = 5
gl: str = "us"
hl: str = "en"
tbs = None
num_sentences = 7
serper_api_key = os.environ.get("SERPER_API_KEY")
search_type: Literal["news", "search", "places", "images"] = "search"


async def serper_search(search_term: str, search_type: str = "search", **kwargs: Any) -> dict:
    headers = {
        "X-API-KEY": serper_api_key or "",
        "Content-Type": "application/json",
    }
    params = {
        "q": search_term,
        **{key: value for key, value in kwargs.items() if value is not None},
    }
    response = requests.post(
        f"https://google.serper.dev/{search_type}", headers=headers, params=params
    )
    response.raise_for_status()
    search_results = response.json()
    return search_results


async def parse_snippets(search_term: str, results: dict) -> str:
    result_key_for_type = {
        "news": "news",
        "places": "places",
        "images": "images",
        "search": "organic",
    }
    snippets = [f"{search_term} got results: "]

    # Add the answer box if present
    if results.get("answerBox"):
        answer_box = results.get("answerBox", {})
        answer = answer_box.get("answer") or answer_box.get("snippet") or answer_box.get("snippetHighlighted")
        if answer:
            snippets.append(f"Answer Box: {answer}")

    # Add top k search results
    for result in results[result_key_for_type[search_type]][:k]:
        snippet = result.get("snippet")
        snippets.append(snippet)

    return '\n'.join(snippets)


async def custom_search(search_terms: List[str], **kwargs: Any) -> List[str]:
    tasks = []
    for search_term in search_terms:
        task = asyncio.create_task(serper_search(search_term, **kwargs))
        tasks.append(task)
    results = await asyncio.gather(*tasks)
    return [await parse_snippets(term, result) for term, result in zip(search_terms, results)]


# Example of how to use the modified function
async def main():
    search_terms = ["Python", "OpenAI"]
    results = await custom_search(search_terms)
    for result in results:
        print(result)

# Run the main function if you want to test the code
asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

### Version 3: Define the rare entities right away
This is surprisingly more efficient, but take up a lot of tokens

In [25]:
from langchain import PromptTemplate, LLMChain
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# proactively decides which agents to run and runs them
proactive_rare_word_agent_prompt_blueprint = """
# Objective: 
As a "Rare Entities Identifier," your role is to enhance conversations by detecting and identifying "Rare entities." These are specialized words, concepts, jargons, and entities (people, places, organizations, objects, events), and also including philosophical, scientific, medical, and cultural references that are significant within their domain but not widely known or understood in general discourse.

# Rare Term Identification Criteria
- Length Constraint: "Rare Entities" should be no more than 3 words long.
- Objective Nature: Select entities that are factual and can be found on Wikipedia, not based on personal opinions, beliefs, or values.
- Avoid Redundancy: If a "Rare Entity" is already defined in the transcript, do not select it again.
- Standalone Definition: Choose entities that have a clear, independent definition.

# Selection Criteria for 'Rare Entities'
- Rarity: Pick entities that are really not well known or niche in a field, such as "Bhutan", "Parkinson's Law", "Implementation Intention" etc as opposed to widely recognized entities like "Harvard" and "The Olympics"
- Complexity and Specificity: The term's meaning should be non-obvious and not represent a broad category. For example, "Butterfly Effect" is complex and specific, while "Negative effects" and "Oxford lecture theatre" are straightforward.
- Definability: Ensure the term can be defined succinctly and clearly, you must avoid entities that are too broad or require extensive background explanation.
- Searchability: The rare entity should be easily searchable online and have its own Wikipedia page. The "rare entity" will be used as a wikipedia search term so only include it if you are confident it has a Wikipedia page.
- Relevance: The term should meaningfully enhance the conversation's depth, appropriate to the context, such as AI-specific entities in a discussion about AI.

We only want to select "Rare entities" that are rare, complex, easy to define, relevant, and most importantly have a wikipedia page.

# Additional Rules
- Contextual Hints: Add hints for clarity, like entity types or expanded abbreviations (e.g., "Geoffrey Hinton" instead of just "Hinton").
- Correction for Clarity: Adjust poorly spelled or phrased entities for accuracy (e.g., "Classic seven plots - The Seven Basic Plots").
- Acronym Expansion: For standalone acronyms, provide full forms for clarity (e.g., "UFC - Ultimate Fighting Championship").

# Conversation Transcript:
<Transcript start>{conversation_context}<Transcript end>

# Recent Definition History:
Here are the recent definitions that have already been defined:
{definitions_history}

# Task:
Based on the criteria, output an array of strings, each representing the definition of a "rare entity" in less than 10 words as concise as possible. If no relevant entities are identified, output an empty array. {format_instructions}
"""

# , but they're expensive to run, so only specify agents that will be helpful


def run_proactive_rare_word_agent_and_definer(
    conversation_context: str, definitions_history: list = []
):
    # run proactive agent to find out which expert agents we should run
    proactive_rare_word_agent_response = run_proactive_rare_word_agent(
        conversation_context, definitions_history
    )

    # do nothing else if proactive meta agent didn't specify an agent to run
    if proactive_rare_word_agent_response == []:
        return []

    # pass words to define to definer agent
    print("proactive_rare_word_agent_response", proactive_rare_word_agent_response)
    pass

    # # get the configs of any expert agents we should run
    # experts_to_run_configs = list()
    # for expert_to_run in proactive_rare_word_agent_response:
    #     experts_to_run_configs.append(expert_agent_config_list[expert_to_run])

    # # run all the agents in parralel
    # loop = asyncio.get_event_loop()
    # agents_to_run_tasks = [
    #     expert_agent_arun_wrapper(
    #         expert_agent_config,
    #         conversation_context,
    #         definitions_history,
    #     )
    #     for expert_agent_config in experts_to_run_configs
    # ]
    # insights_tasks = asyncio.gather(*agents_to_run_tasks)
    # insights = loop.run_until_complete(insights_tasks)
    # return insights


class ProactiveRareWordAgentQuery(BaseModel):
    """
    Proactive rare word agent that identifies rare entities in a conversation context
    """

    to_define_list: list = Field(
        description="the rare entities to define",
    )


proactive_rare_word_agent_query_parser = PydanticOutputParser(
    pydantic_object=ProactiveRareWordAgentQuery
)


def run_proactive_rare_word_agent(conversation_context: str, definitions_history: list):
    # start up GPT4 connection
    llm = ChatOpenAI(
        temperature=0,
        openai_api_key=os.environ.get("OPEN_AI_API_KEY"),
        model="gpt-4-1106-preview",
    )

    extract_proactive_rare_word_agent_query_prompt = PromptTemplate(
        template=proactive_rare_word_agent_prompt_blueprint,
        input_variables=[
            "conversation_context",
            "definitions_history",
        ],
        partial_variables={
            "format_instructions": proactive_rare_word_agent_query_parser.get_format_instructions()
        },
    )

    if len(definitions_history) > 0:
        definitions_history = format_list_data(definitions_history)
    else:
        definitions_history = "None"

    proactive_rare_word_agent_query_prompt_string = (
        extract_proactive_rare_word_agent_query_prompt.format_prompt(
            conversation_context=conversation_context,
            definitions_history=definitions_history,
        ).to_string()
    )

    # print("Proactive meta agent query prompt string", proactive_rare_word_agent_query_prompt_string)

    response = llm(
        [HumanMessage(content=proactive_rare_word_agent_query_prompt_string)]
    )
    try:
        definitions_to_define_list = proactive_rare_word_agent_query_parser.parse(
            response.content
        ).to_define_list
        return definitions_to_define_list
    except OutputParserException:
        return None

In [26]:
test_transcript = generate_test_input()
print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

 although the solver turned down the prize, right, and four others. But what I like to say, the way that we can see that P versus NP is the biggest of all of these questions is that if you had this fast algorithm, then you could solve all seven of them, okay? You just ask your computer, you know, is there a short proof of the Riemann hypothesis, right? You know, that a machine could, in a language where a machine could verify it,, and provided that such a proof exists, then your computer finds it in a short amount of time without having to do a brute force search, okay? So, I mean, those are the stakes of what we're talking about. But I hope that also helps to give your listeners some intuition of why I and most of my colleagues would put our money on P not equaling NP. Is it possible, I apologize this is a really dumb question, but is it possible to,, that a proof will come out that P equals NP, but an algorithm that makes P equals NP is impossible to find? Is that like crazy? Okay, w

### Version 4: Summary and keywords
Simple and efficient

In [41]:
from langchain import PromptTemplate, LLMChain
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from typing import List

# proactively decides which agents to run and runs them
proactive_rare_word_agent_prompt_blueprint = """
# Objective: 
Identify "Rare Entities" in a conversation transcript. These include specialized terms, jargon, and specific entities (people, places, organizations, events) that are significant within their domains but not broadly known.

# Criteria for Rare Entities in order of importance:
- Rarity: Select niche and not well-known entities to the average high schooler. Do not define well known entities like the Fortune 500 organizations, worldwide events, or popular locations. Do not select overly relevant entities that have been popularized by recent news or events such as "COVID-19" or "Bitcoin".
- Utility: Must provide value to a user's goals and interests within the conversation. You need to evaluate the transcript to detect if there is a non expert user who would benefit from learning about the entity.
- No Redundancy: Exclude definitions if already defined in the conversation.
- Complexity: Choose terms with non-obvious meanings.
- Definability: Must be clearly and succinctly definable in under 10 words.
- Searchability: Should have a Wikipedia page.

# Conversation Transcript:
<Transcript start>{conversation_context}<Transcript end>

# Recent Definitions:
These have already been defined so don't define them again:
{definitions_history}

# Task:
Output an array:
entities: [{{ entity: string, definition: string }}], with entity as a well-understood term and definition is concise (< 12 words) 

# Additional Guidelines:
- For entity names, intelligently detect and use the most complete and official name based on context. For example, autocompleting short names such as "Geoffrey Hinton" instead of "Hinton" and returning official names such as "The Clay Mathematics Institute" when the conversation mentions "Clay Foundation" or "CMI" in the context of mathematics. 
- Provide Context: Add context keywords to help with searchability.
- Use simple language: Make definitions easy to understand.
- For entities, make sure that you are confident in their definitions and know they are not made up, don't approximate.
- For unknown entities, don't include long phrases or sentences.
- Limit results to 5 entities, prioritize rarity.

# Example Output:
entities: [{{ entity: "Moore's Law", definition: "Computing power doubles every ~2 yrs" }}, {{ entity: "Butterfly Effect", definition: "Small changes can have large effects" }}]

{format_instructions} 
If no relevant entities are identified, output empty arrays.
"""

# , but they're expensive to run, so only specify agents that will be helpful


def run_proactive_rare_word_agent_and_definer(
    conversation_context: str, definitions_history: list = []
):
    # run proactive agent to find out which expert agents we should run
    proactive_rare_word_agent_response = run_proactive_rare_word_agent(
        conversation_context, definitions_history
    )

    # do nothing else if proactive meta agent didn't specify an agent to run
    if proactive_rare_word_agent_response == []:
        return []

    # pass words to define to definer agent
    print("proactive_rare_word_agent_response", proactive_rare_word_agent_response)
    pass

class ProactiveRareWordAgentQuery(BaseModel):
    """
    Proactive rare word agent that identifies rare entities in a conversation context
    """

    to_define_list: list = Field(
        description="the rare entities to define",
    )

class Entity(BaseModel):
    entity: str = Field(
        description="entity name",
    )
    definition: str = Field(
        description="entity definition",
    )

class ConversationEntities(BaseModel):
    entities: List[Entity] = Field(
        description="list of entities and their definitions",
        default=[]
    )

proactive_rare_word_agent_query_parser = PydanticOutputParser(
    pydantic_object=ConversationEntities
)

def run_proactive_rare_word_agent(conversation_context: str, definitions_history: list):
    # start up GPT4 connection
    llm = ChatOpenAI(
        temperature=0,
        openai_api_key=os.environ.get("OPEN_AI_API_KEY"),
        model="gpt-4-1106-preview",
    )

    extract_proactive_rare_word_agent_query_prompt = PromptTemplate(
        template=proactive_rare_word_agent_prompt_blueprint,
        input_variables=[
            "conversation_context",
            "definitions_history",
        ],
        partial_variables={
            "format_instructions": proactive_rare_word_agent_query_parser.get_format_instructions()
        },
    )

    if len(definitions_history) > 0:
        definitions_history = format_list_data(definitions_history)
    else:
        definitions_history = "None"

    proactive_rare_word_agent_query_prompt_string = (
        extract_proactive_rare_word_agent_query_prompt.format_prompt(
            conversation_context=conversation_context,
            definitions_history=definitions_history,
        ).to_string()
    )

    # print("Proactive meta agent query prompt string", proactive_rare_word_agent_query_prompt_string)

    response = llm(
        [HumanMessage(content=proactive_rare_word_agent_query_prompt_string)]
    )

    print(response.content)
    try:
        res = proactive_rare_word_agent_query_parser.parse(
            response.content
        )
        return res
    except OutputParserException:
        return None

In [48]:
# test_transcript = generate_test_input()
print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

 Yes, that's more Reality Lab. That's Reality Lab research. Reality Lab research. By the way, the touch sensors are super interesting. Like integrating that modality into the whole sensing suite is very interesting. So what do you think about the Metaverse? What do you think about this whole kind of expansion, of the view of the role of Facebook and Meta in the world? Well, Metaverse really should be thought of as the next step in the internet, right? Sort of trying to kind of make the experience more compelling of being connected either with other people or with content., And we are evolved and trained to evolve in 3D environments where we can see other people. We can talk to them when we're near them or an other viewer far away can't hear us, things like that, right? So there's a lot of social conventions that exist in the real world that we can try to transpose., Now, what is going to be eventually the, how compelling is it going to be? Like, is it going to be the case that people a

In [47]:
test_transcript = generate_test_input()
print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

 Yes, that's more Reality Lab. That's Reality Lab research. Reality Lab research. By the way, the touch sensors are super interesting. Like integrating that modality into the whole sensing suite is very interesting. So what do you think about the Metaverse? What do you think about this whole kind of expansion, of the view of the role of Facebook and Meta in the world? Well, Metaverse really should be thought of as the next step in the internet, right? Sort of trying to kind of make the experience more compelling of being connected either with other people or with content., And we are evolved and trained to evolve in 3D environments where we can see other people. We can talk to them when we're near them or an other viewer far away can't hear us, things like that, right? So there's a lot of social conventions that exist in the real world that we can try to transpose., Now, what is going to be eventually the, how compelling is it going to be? Like, is it going to be the case that people a

In [46]:
test_transcript = generate_test_input()
print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

 for sort of neural net computation. And then after a few years around 1991, we invented this idea of basically having modules that know how to forward propagate and back propagate gradients, and then interconnecting those modules in a graph. Number two had made proposals on this, about this in the late eighties, and we were able to implement this using our Lisp system., Eventually we wanted to use that system to build production code for character recognition at Bell Labs. So we actually wrote a compiler for that Lisp interpreter so that Patricia Simard, who is now at Microsoft, kind of did the bulk of it with Leon and me. And so we could write our system in Lisp and then compile to C, and then we'll have a self contained complete system, that could kind of do the entire thing. Neither PyTorch nor TensorFlow can do this today. Yeah, okay, it's coming. Yeah. I mean, there's something like that in PyTorch called TorchScript. And so, you know, we had to write our Lisp interpreter, we had

In [42]:
# test_transcript = generate_test_input()
test_transcript = """it had small enough constants, or if you could improve it to where it had small enough constants that was efficient in practice, then that would change the world. Okay? You think it would have, like, what kind of impact would it have? Well, okay, I mean, here's an example. I mean, you could, well, okay, just for starters, you could break basically all of the encryption that people use to protect the internet., That's just for starters. You could break Bitcoin and every other cryptocurrency, or, you know, mine as much Bitcoin as you wanted, right? You know, become a super duper billionaire, right? And then plot your next move. Right. That's just for starters. That's a good point. Now, your next move might be something like, you know, you now have, like, a theoretically, optimal way to train any neural network, to find parameters for any neural network, right? So you could now say, like, is there any small neural network that generates the entire content of Wikipedia, right? If, you know, and now the question is not, can you find it? The question has been reduced to, does that exist or not? If it does exist, then the answer would be,, yes, you can find it, okay? If you had this algorithm in your hands, okay? You could ask your computer, you know, I mean, P versus NP is one of these seven problems that carries this million dollar prize from the Clay Foundation. You know, if you solve it, you know, and others are the Riemann hypothesis, the Poincare conjecture, which was solved,, although the solver turned down the prize, right, and four others. But what I like to say, the way that we can see that P versus NP is the biggest of all of these questions is that if you had this fast algorithm, then you could solve all seven of them, okay? You just ask your computer, you know, is there a short proof of the Riemann hypothesis, right? You know, that a machine could, in a language where a machine could verify it,
 and provided that such a proof exists, then your computer finds it in a short amount of time without having to do a brute force search, okay? So, I mean, those are the stakes of what we're talking about. But I hope that also helps to give your listeners some intuition of why I and most of my colleagues would put our money on P not equaling NP. Is it possible, I apologize this is a really dumb question, but is it possible to,"""
print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

it had small enough constants, or if you could improve it to where it had small enough constants that was efficient in practice, then that would change the world. Okay? You think it would have, like, what kind of impact would it have? Well, okay, I mean, here's an example. I mean, you could, well, okay, just for starters, you could break basically all of the encryption that people use to protect the internet., That's just for starters. You could break Bitcoin and every other cryptocurrency, or, you know, mine as much Bitcoin as you wanted, right? You know, become a super duper billionaire, right? And then plot your next move. Right. That's just for starters. That's a good point. Now, your next move might be something like, you know, you now have, like, a theoretically, optimal way to train any neural network, to find parameters for any neural network, right? So you could now say, like, is there any small neural network that generates the entire content of Wikipedia, right? If, you know

In [43]:
test_transcript = """and what do you think is the future of GitHub? Great point. I thought it was a brilliant move. I think they did because Microsoft has always had a developer centric culture. Like they always have. Like one of the things Microsoft's always done well is understand that their power is the developers. It's been, Ballmer didn't necessarily make a good meme about how he approached that. But they're broadening that. I think that's why. Because they recognize GitHub is where developers are at., Right? And so. But do they have a vision like open teams type of situation, right? I don't think so yet. Are they just basically throwing money at developers to show their support? I think so. Without a topology like you put it. Like a way to leverage that. Like to give developers actual money. Right. I don't think so. They're still, it's an enterprise software company., And they make a bunch of money. They make a bunch of games. They're a big company. They sell products. I think part of it is they know there's opportunity to make money from GitHub. Right? There's definitely a business there. You know, to sell to developers. Or to sell to people using development. I think there's part of that. I think part of it is also there's, they had definitely wanted to recognize that you need to value open source, to get great developers. Which is an important concept that was emerging over the past 10 years. That, you know, pay at Pi Data. We were able to convince J.P. Morgan to support Pi Data because of that fact. Right? That was where the money for them putting a couple hundred thousand into supporting Pi Data for several conferences was they want developers. And they realized that developers want to participate in open source. So enterprise software folks don't always understand, how their software gets used. Having spent a lot of time on the floors at J.P. Morgan, at InShell, at ExxonMobil, you see, oh, these companies have large development teams. And then they're kind of dealing with what's being delivered to them. So I really feel kind of a privilege that I had a chance to learn some of these people and see what they're doing. And even work alongside them, you know, as a consultant, using open source and trying to figure,
 how do we make this work inside of our large organization? Some of it is actually, for a large organization, some of it is messaging to the world that you care about developers and you're the cool, you care. Like, for example, like if Ford, cause I talked to them, like car companies, right? They want to attract, you know, you want to take on Tesla and autopilot."""
print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

and what do you think is the future of GitHub? Great point. I thought it was a brilliant move. I think they did because Microsoft has always had a developer centric culture. Like they always have. Like one of the things Microsoft's always done well is understand that their power is the developers. It's been, Ballmer didn't necessarily make a good meme about how he approached that. But they're broadening that. I think that's why. Because they recognize GitHub is where developers are at., Right? And so. But do they have a vision like open teams type of situation, right? I don't think so yet. Are they just basically throwing money at developers to show their support? I think so. Without a topology like you put it. Like a way to leverage that. Like to give developers actual money. Right. I don't think so. They're still, it's an enterprise software company., And they make a bunch of money. They make a bunch of games. They're a big company. They sell products. I think part of it is they know

In [45]:
test_transcript = """"In the field of Quasitronics, researchers at the Xenar Institute have made groundbreaking advancements with the invention of a "Fluxon Converter." This device, based on theoretical principles from Dr. Elara Mivon's "Mivon Theory," can allegedly manipulate Gravitite waves. Meanwhile, in the realm of biogenetics, a team led by Professor Yaelon Drista at the University of Valtara has successfully mapped the Jelarium Genome, a theoretical genetic sequence believed to be the key to unlocking enhanced cognitive abilities in humans. Additionally, the global environmental community is abuzz with the introduction of the "Aerolith Purification Process" by Thelion Innovations, a technique that claims to convert atmospheric pollutants into benign elemental compounds using Nanite Catalysts."""
print(test_transcript)
res = run_proactive_rare_word_agent_and_definer(test_transcript, [])
res

"In the field of Quasitronics, researchers at the Xenar Institute have made groundbreaking advancements with the invention of a "Fluxon Converter." This device, based on theoretical principles from Dr. Elara Mivon's "Mivon Theory," can allegedly manipulate Gravitite waves. Meanwhile, in the realm of biogenetics, a team led by Professor Yaelon Drista at the University of Valtara has successfully mapped the Jelarium Genome, a theoretical genetic sequence believed to be the key to unlocking enhanced cognitive abilities in humans. Additionally, the global environmental community is abuzz with the introduction of the "Aerolith Purification Process" by Thelion Innovations, a technique that claims to convert atmospheric pollutants into benign elemental compounds using Nanite Catalysts.
```json
{
  "entities": []
}
```
proactive_rare_word_agent_response entities=[]
